In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
combined_data = pd.read_csv('tvshows.csv')
combined_data['features'] = combined_data['overview'].fillna('').astype(str) + ' ' + combined_data['vote_average'].fillna('').astype(str) + ' ' + combined_data['popularity'].fillna('').astype(str) +  ' ' + combined_data['original_language'].fillna('').astype(str)

In [3]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(combined_data['features'])
tfidf_models = (tfidf, tfidf_matrix)

In [4]:
nn_model = NearestNeighbors(n_neighbors=6, algorithm='auto')
nn_model.fit(tfidf_models[1])
nn_models = nn_model

In [5]:
def get_recommendations_with_overviews(show_name, tfidf_models=tfidf_models, nn_models=nn_models, combined_data=combined_data):
    tfidf, tfidf_matrix = tfidf_models
    nn_model = nn_models
    show_features = combined_data[combined_data['name'] == show_name]['features'].iloc[0]
    show_tfidf = tfidf.transform([show_features])
    distances, indices = nn_model.kneighbors(show_tfidf)
    show_indices = indices.flatten()[1:]
    recommendations_with_overviews = []
    for idx in show_indices:
        show_name = combined_data['name'].iloc[idx]
        show_overview = combined_data['overview'].iloc[idx]
        recommendations_with_overviews.append(show_name)
    return recommendations_with_overviews

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommendations', methods=['POST'])
def recommendations():
    movie_name = request.form['movie_name']
    recommendations = get_recommendations_with_overviews(movie_name)
    return render_template('recommendations.html', recommendations=recommendations)

if __name__ == '__main__':
    app.run(port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2024 13:39:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2024 13:39:47] "POST /recommendations HTTP/1.1" 200 -
